Hyperparameter tuning

In [1]:
from CBF.cbf_recommender import cbf_recommender
from utils.evaluator.evaluator import evaluator
from utils.submission_helper import submission_helper
from utils.data_manager.data_manager import data_manager
from utils.data_manager.splitter import splitter

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

Now I'm trying to discover the best similarity heuristic and also if there is a dependency on the k parameter

In [2]:
training, validation, test = splitter().get_train_evaluation_test(percentage_of_training_data=0.6,
                                                                  percentage_of_validation_data=0.2)

In [3]:
k = [100, 250, 400]
similarities = ['pearson', 'cosine', 'adjusted', 'asymmetric', 'jaccard', 'tanimoto',
                             'dice', 'tversky']
map_vectors = []
for chosen_k in k:
    map_vector = []
    for chosen_similarity in similarities:
        recommender_ = cbf_recommender()
        recommender_.fit(training, k=chosen_k, normalize=True, similarity=chosen_similarity)
        map_vector.append(evaluator.evaluate_already_trained(recommender=recommender_)[2])
    map_vectors.append(map_vector)
index = 0
for vector in map_vectors:
    print("k: ", k[index], "map: ", vector, "similarities: ", similarities)
    index+=1

Similarity column 25975 ( 100 % ), 1571.95 column/sec, elapsed time 0.28 min
Similarity column 25975 ( 100 % ), 1299.09 column/sec, elapsed time 0.33 min
Similarity column 25975 ( 100 % ), 1507.73 column/sec, elapsed time 0.29 min
Similarity column 25975 ( 100 % ), 1416.92 column/sec, elapsed time 0.31 min
Similarity column 25975 ( 100 % ), 1363.16 column/sec, elapsed time 0.32 min
Similarity column 25975 ( 100 % ), 1209.59 column/sec, elapsed time 0.36 min
Similarity column 25975 ( 100 % ), 1271.35 column/sec, elapsed time 0.34 min
Similarity column 25975 ( 100 % ), 882.18 column/sec, elapsed time 0.49 min
Similarity column 25975 ( 100 % ), 1048.56 column/sec, elapsed time 0.41 min
Similarity column 25975 ( 100 % ), 1207.63 column/sec, elapsed time 0.36 min
Similarity column 25975 ( 100 % ), 1128.90 column/sec, elapsed time 0.38 min
Similarity column 25975 ( 100 % ), 1389.93 column/sec, elapsed time 0.31 min
Similarity column 25975 ( 100 % ), 1293.44 column/sec, elapsed time 0.33 min


In [ ]:
k_tested = [i for i in range(30,400,30)]
k1 = [i for i in range(450, 700, 60)]
k_tested = k_tested + k1
map_=[]
for k in k_tested:
    recommender_k = cbf_recommender()
    recommender_k.fit(training, k=k, normalize=True, similarity='jaccard')
    map_.append(evaluator.evaluate_already_trained(recommender=recommender_k)[2])
plt.plot(k_tested, map_)
plt.ylabel('MAP')
plt.xlabel('K')
plt.show()

Similarity column 25975 ( 100 % ), 1227.78 column/sec, elapsed time 0.35 min
Similarity column 25975 ( 100 % ), 1296.03 column/sec, elapsed time 0.33 min
Similarity column 25975 ( 100 % ), 1248.38 column/sec, elapsed time 0.35 min
Similarity column 25975 ( 100 % ), 1315.25 column/sec, elapsed time 0.33 min
Similarity column 25975 ( 100 % ), 1302.00 column/sec, elapsed time 0.33 min
Similarity column 25975 ( 100 % ), 1225.08 column/sec, elapsed time 0.35 min
Similarity column 25975 ( 100 % ), 1148.83 column/sec, elapsed time 0.38 min
Similarity column 25975 ( 100 % ), 1144.17 column/sec, elapsed time 0.38 min
Similarity column 25975 ( 100 % ), 1326.88 column/sec, elapsed time 0.33 min
Similarity column 25975 ( 100 % ), 1124.30 column/sec, elapsed time 0.39 min
Similarity column 25975 ( 100 % ), 1146.19 column/sec, elapsed time 0.38 min
Similarity column 25975 ( 100 % ), 1302.20 column/sec, elapsed time 0.33 min
Similarity column 25975 ( 100 % ), 995.05 column/sec, elapsed time 0.44 min


In [ ]:
k = k_tested[np.argmax(map_)]
k

In [ ]:
shrinks = [i for i in range(6,400, 30)]
map_=[]
for s in shrinks:
    recommender_shrink = cbf_recommender()
    recommender_shrink.fit(training, k=90, shrink=s, normalize=True, similarity='jaccard')
    map_.append(evaluator.evaluate_already_trained(recommender=recommender_shrink)[2])
plt.plot(shrinks, map_)
plt.ylabel('MAP')
plt.xlabel('shrink')
plt.show()

In [ ]:
shrink = shrinks[np.argmax(map_)]
shrink


In [ ]:
shrinks = [i for i in range(10,60, 5)]
map_=[]
for s in shrinks:
    recommender_s = cbf_recommender()
    recommender_s.fit(training, k=90, shrink=s, normalize=True, similarity='jaccard')
    map_.append(evaluator.evaluate_already_trained(recommender=recommender_s)[2])
plt.plot(shrinks, map_)
plt.ylabel('MAP')
plt.xlabel('shrink')
plt.show()

In [ ]:
shrink = shrinks[np.argmax(map_)]
shrink

In [ ]:
from utils.IR_feature_weighting import okapi_BM_25
ICM_BM25 = data_manager().get_icm().astype(np.float32)
ICM_BM25 = okapi_BM_25(ICM_BM25)
ICM_BM25 = ICM_BM25.tocsc()
recommender_BM = cbf_recommender(icm=ICM_BM25)
recommender_BM.fit(training_set=training, shrink=60, k=90, similarity='jaccard')
map_ = evaluator.evaluate_already_trained(recommender=recommender_BM)[2]
print(map_)

In [ ]:
from utils.IR_feature_weighting import TF_IDF
import scipy.sparse as sparse
icm = data_manager().get_icm()
num_tot_items = icm.shape[0]
items_per_feature = (icm>0).sum(axis=0)
IDF = np.array(np.log(num_tot_items/items_per_feature))[0]
icm_idf = icm.copy()
col_nnz = np.diff(sparse.csc_matrix(icm_idf).indptr)
icm_idf.data *= np.repeat(IDF, col_nnz)
recommender_idf = cbf_recommender(icm=icm_idf)
recommender_idf.fit(training_set=training, shrink=60, k =90, similarity='jaccard')
map_ = evaluator.evaluate_already_trained(recommender=recommender_idf)[2]
print(map_)

